# Dylan Forsythe

## Research Interests

Since pursuing a degree in data science, I am constantly being introduced to various ways of observing and exploring data, tables, charts, and graphs. I very much enjoy the visualization of dataframes by turning them into graphs because doing so helps me understand the data. By learning how to customize these graphs, theres an almost unlimited amount of stories that can be created. That is where the spark of my research interests lay. I wish to find the stories in the data that are not apparent without the steps involved with data analysis.
For this project, my interest is in learning about the popularities of some of the the sports over the past 30 years. So far, I'm mostly interested in winter olympic sports like snowboarding because of how 'new' it is relative to the other sports, and figure skating simply because I think its pretty. I am hoping to find information by looking at how the popularity of a sport changes over time. Perhaps, if I am able to learn enough analysis and statistics techniques, I may be able to find how the popularity of one sport influences or affects the popularity of another. This question may have to be dropped as it may not be within the scope of this course.

## Research Question
**How do the popularities of each sport (i.e. amount of participants/competitors) change over time for each sport?**

In [2]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Set the base theme for seaborn graphs
sns.set_theme(style='whitegrid',font_scale=1.25)

In [4]:
# Read raw data and save into seperate df's
OlympicResultsDF = pd.read_csv('../../data/raw/Olympic_Summer_Winter_Games_1986-2022/olympic_results.csv')
OlympicHostsDF = pd.read_csv('../../data/raw/Olympic_Summer_Winter_Games_1986-2022/olympic_hosts.csv')
OlympicMedalsDF = pd.read_csv('../../data/raw/Olympic_Summer_Winter_Games_1986-2022/olympic_medals.csv')
OlympicAthletesDF = pd.read_csv('../../data/raw/Olympic_Summer_Winter_Games_1986-2022/olympic_athletes.csv')

In [46]:
# Check the shape, and heads of each dataframe to get an idea of what information each one contains
print(OlympicResultsDF.shape)
OlympicResultsDF.head()

(162804, 15)


,discipline_title,event_title,slug_game,participant_type,medal_type,athletes,rank_equal,rank_position,country_name,country_code,country_3_letter_code,athlete_url,athlete_full_name,value_unit,value_type
0,Curling,Mixed Doubles,beijing-2022,GameTeam,GOLD,"[('Stefania CONSTANTINI', 'https://olympics.co...",False,1,Italy,IT,ITA,NaN,NaN,NaN,NaN
1,Curling,Mixed Doubles,beijing-2022,GameTeam,SILVER,"[('Kristin SKASLIEN', 'https://olympics.com/en...",False,2,Norway,NO,NOR,NaN,NaN,NaN,NaN
2,Curling,Mixed Doubles,beijing-2022,GameTeam,BRONZE,"[('Almida DE VAL', 'https://olympics.com/en/at...",False,3,Sweden,SE,SWE,NaN,NaN,NaN,NaN
3,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,"[('Jennifer DODDS', 'https://olympics.com/en/a...",False,4,Great Britain,GB,GBR,NaN,NaN,NaN,NaN
4,Curling,Mixed Doubles,beijing-2022,GameTeam,NaN,"[('Rachel HOMAN', 'https://olympics.com/en/ath...",False,5,Canada,CA,CAN,NaN,NaN,NaN,NaN


In [47]:
print(OlympicHostsDF.shape)
OlympicHostsDF.head()

(53, 7)


,game_slug,game_end_date,game_start_date,game_location,game_name,game_season,game_year
0,beijing-2022,2022-02-20T12:00:00Z,2022-02-04T15:00:00Z,China,Beijing 2022,Winter,2022
1,tokyo-2020,2021-08-08T14:00:00Z,2021-07-23T11:00:00Z,Japan,Tokyo 2020,Summer,2020
2,pyeongchang-2018,2018-02-25T08:00:00Z,2018-02-08T23:00:00Z,Republic of Korea,PyeongChang 2018,Winter,2018
3,rio-2016,2016-08-21T21:00:00Z,2016-08-05T12:00:00Z,Brazil,Rio 2016,Summer,2016
4,sochi-2014,2014-02-23T16:00:00Z,2014-02-07T04:00:00Z,Russian Federation,Sochi 2014,Winter,2014


In [48]:
print(OlympicMedalsDF.shape)
OlympicMedalsDF.head()

(21697, 12)


,discipline_title,slug_game,event_title,event_gender,medal_type,participant_type,participant_title,athlete_url,athlete_full_name,country_name,country_code,country_3_letter_code
0,Curling,beijing-2022,Mixed Doubles,Mixed,GOLD,GameTeam,Italy,https://olympics.com/en/athletes/stefania-cons...,Stefania CONSTANTINI,Italy,IT,ITA
1,Curling,beijing-2022,Mixed Doubles,Mixed,GOLD,GameTeam,Italy,https://olympics.com/en/athletes/amos-mosaner,Amos MOSANER,Italy,IT,ITA
2,Curling,beijing-2022,Mixed Doubles,Mixed,SILVER,GameTeam,Norway,https://olympics.com/en/athletes/kristin-skaslien,Kristin SKASLIEN,Norway,NO,NOR
3,Curling,beijing-2022,Mixed Doubles,Mixed,SILVER,GameTeam,Norway,https://olympics.com/en/athletes/magnus-nedreg...,Magnus NEDREGOTTEN,Norway,NO,NOR
4,Curling,beijing-2022,Mixed Doubles,Mixed,BRONZE,GameTeam,Sweden,https://olympics.com/en/athletes/almida-de-val,Almida DE VAL,Sweden,SE,SWE


In [49]:
print(OlympicAthletesDF.shape)
OlympicAthletesDF.head()

(75904, 7)


,athlete_url,athlete_full_name,games_participations,first_game,athlete_year_birth,athlete_medals,bio
0,https://olympics.com/en/athletes/cooper-woods-...,Cooper WOODS-TOPALOVIC,1,Beijing 2022,2000.0,NaN,NaN
1,https://olympics.com/en/athletes/elofsson,Felix ELOFSSON,2,PyeongChang 2018,1995.0,NaN,NaN
2,https://olympics.com/en/athletes/dylan-walczyk,Dylan WALCZYK,1,Beijing 2022,1993.0,NaN,NaN
3,https://olympics.com/en/athletes/olli-penttala,Olli PENTTALA,1,Beijing 2022,1995.0,NaN,NaN
4,https://olympics.com/en/athletes/reikherd,Dmitriy REIKHERD,1,Beijing 2022,1989.0,NaN,NaN


## Thoughts
- I am deciding to use the OlympicResultsDF as my main database source because it contains the largest amount of information and has columns for most of the information I will need.  
- I will probably need to merge a few columns from other dataframes to this one

In [116]:
# I want to create a dataframe that contains all the information I would like from the above four dataframes
# Then I should be able to create some visualizations to learn more about what the dataframe contains

Research_df = OlympicResultsDF
Research_df = Research_df.drop(columns=['country_name','country_code','country_3_letter_code',
                                        'athlete_url','rank_equal','rank_position','athlete_full_name',
                                        'value_unit','value_type','medal_type'],axis=1)
Research_df = Research_df.rename(columns= {'discipline_title':'Sport Name',
                                           'event_title':'Event Name',
                                           'slug_game':'Location',
                                           'athletes':'Athlete Name'})
Research_df = Research_df.dropna(subset='Athlete Name').reset_index()
Research_df = Research_df.drop(columns=['index'])


Research_df

,Sport Name,Event Name,Location,participant_type,Athlete Name
0,Curling,Mixed Doubles,beijing-2022,GameTeam,"[('Stefania CONSTANTINI', 'https://olympics.co..."
1,Curling,Mixed Doubles,beijing-2022,GameTeam,"[('Kristin SKASLIEN', 'https://olympics.com/en..."
2,Curling,Mixed Doubles,beijing-2022,GameTeam,"[('Almida DE VAL', 'https://olympics.com/en/at..."
3,Curling,Mixed Doubles,beijing-2022,GameTeam,"[('Jennifer DODDS', 'https://olympics.com/en/a..."
4,Curling,Mixed Doubles,beijing-2022,GameTeam,"[('Rachel HOMAN', 'https://olympics.com/en/ath..."
...,...,...,...,...,...
7971,Figure skating,Ice dancing mixed,innsbruck-1976,GameTeam,"[('Isabella RIZZI', 'https://olympics.com/en/a..."
7972,Figure skating,Ice dancing mixed,innsbruck-1976,GameTeam,"[('Judi GENOVESI', 'https://olympics.com/en/at..."
7973,Figure skating,Ice dancing mixed,innsbruck-1976,GameTeam,"[('Stefania BERTELE', 'https://olympics.com/en..."
7974,Figure skating,Ice dancing mixed,innsbruck-1976,GameTeam,"[('Susan KELLEY', 'https://olympics.com/en/ath..."
